In [1]:
import numpy as np
from functools import lru_cache
from timing import timed, compare
from matplotlib import pyplot as plt

# Динамическое программирование

Динамическое программирование в теории управления и теории вычислительных систем — способ решения сложных задач путём разбиения их на более простые подзадачи. Он применим к задачам с оптимальной подструктурой, выглядящим как набор перекрывающихся подзадач, сложность которых чуть меньше исходной. В этом случае время вычислений, по сравнению с «наивными» методами, можно значительно сократить.

Ключевая идея в динамическом программировании достаточно проста. Как правило, чтобы решить поставленную задачу, требуется решить отдельные части задачи (подзадачи), после чего объединить решения подзадач в одно общее решение. Часто многие из этих подзадач одинаковы. Подход динамического программирования состоит в том, чтобы решить каждую подзадачу только один раз, сократив тем самым количество вычислений. Это особенно полезно в случаях, когда число повторяющихся подзадач экспоненциально велико. 

- Вместо исходной задачи решается множество перекрывающихся подзадач. Ответы для подзадач хранятся в таблице.
- Динамическое программирование назад (или сверху вниз): рекурсивно от больших задач к меньшим.
- Динамическое программирование вперед (или снизу вверх): итеративно от меньших задач к большим.
- Для некоторых задач можно уменьшить используемую память, проанализировав структуру таблицы.

# Расстояние редактирования
Вычислите расстояние редактирования двух данных непустых строк длины не более $10^2$, содержащих строчные буквы латинского алфавита.

Sample Input 1:
```
ab
ab
```
Sample Output 1:
```
0
```
Sample Input 2:
```
short
ports
```
Sample Output 2:
```
3
```
### Функции из лекции

In [2]:
import random
import sys
from functools import lru_cache

def edit_distance_iters(s1,s2):
    m, n = len(s1), len(s2)
    d = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j    
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            d[i][j] = min(d[i][j - 1] + 1,
                      d[i - 1][j] + 1,
                      d[i - 1][j - 1] + (s1[i-1] != s2[j - 1]))
    return d[m][n]

def edit_distance_iters_V2(s1,s2):
    m, n = len(s1), len(s2)
    if m < n:
        return edit_distance_iters_V2(s2,s1)
    
    prev = list(range(n + 1))
    for i, ch1 in enumerate(s1, 1):
        curr = [i] #0(min(m,n))
        for j, ch2 in enumerate(s2, 1):
            curr.append(min(curr[-1] + 1,
                      prev[j] + 1,
                      prev[j - 1] + (ch1 != ch2)))
        prev = curr
    return prev[n]

def edit_distance_recurs(s1,s2):
    @lru_cache(maxsize=None)
    def d(i,j):
        if i == 0 or j == 0:
            return max(i,j)
        else:
            return min(d(i, j - 1) + 1,
                      d(i - 1, j) + 1,
                      d(i - 1, j - 1) + (s1[i-1] != s2[j - 1]))
    return d(len(s1), len(s2))

### Тесты

In [3]:
def test(func_edit_distance, n_iter=10):
    for i in range(n_iter):
        lenght = random.randint(0,100)
        s = "".join(random.choice("01") for _ in range(lenght))
        
        assert func_edit_distance(s, "") == func_edit_distance("", s) == len(s)
        assert func_edit_distance(s,s) == 0
    assert func_edit_distance("ab", "ab") == 0
    assert func_edit_distance("short", "ports") == 3

In [4]:
func_dic = {
    'edit_distance_recurs':edit_distance_recurs,
    'edit_distance_iters':edit_distance_iters,
    'edit_distance_iters_V2':edit_distance_iters_V2
}
for name in func_dic:
    try:
        test(func_dic[name])
        print(name, 'отработал без ошибок')
    except:
        print(name, 'отработал с ошибками')       

edit_distance_recurs отработал без ошибок
edit_distance_iters отработал без ошибок
edit_distance_iters_V2 отработал без ошибок


### Мои функции

In [5]:
def diff(char1, char2):
    return 0 if char1 == char2 else 1
def edit_dist_td(i, j):
    if i == 0:
        D[i,j] = j
    elif j == 0:
        D[i,j] = i
    else:
        ins = edit_dist_td(i, j - 1) + 1
        delete = edit_dist_td(i - 1, j) + 1
        sub = edit_dist_td(i - 1, j - 1) + diff(A[i-1], B[j-1])
        D[i,j] = min(ins,delete,sub)
    return D[i,j]

def edit_dist_td_iters(A, B):
    n, m = len(A), len(B)
    D = np.full((n,m), np.inf)
    for i in range(n):
        D[i,0] = i
    for j in range(m):
        D[0,j] = j
    for i in range(1,n):
        for j in range(1,m):
            d = diff(A[i-1], B[j-1])
            D[i,j] = min(D[i-1,j],D[i,j-1]+1,D[i-1,j-1]+d)
    return D[n,m]

### Тесты

In [6]:
A = 'хлеб'
B = 'пиво'
D = np.full((len(A) + 1, len(B) + 1), np.inf)
dist = edit_dist_td(len(A), len(B))
print('Расстояние между', A, 'и', B, 'равно', dist)
print(D)

Расстояние между хлеб и пиво равно 4.0
[[0. 1. 2. 3. 4.]
 [1. 1. 2. 3. 4.]
 [2. 2. 2. 3. 4.]
 [3. 3. 3. 3. 4.]
 [4. 4. 4. 4. 4.]]
